In [ ]:
import sys
sys.path.insert(0, "/notebooks/pipenv")
sys.path.insert(0, "/notebooks/nebula3_database")
from torch.nn.functional import softmax
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch
import random
import csv

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
device = "cuda:0"
model = model.to(device)

In [ ]:
stories = []
with open("1000ROC.csv") as file:
    tsv_file = csv.reader(file, delimiter=",")
    for line in tsv_file:
        #print(line)
        line.pop(0)
        #print(line)
        line.pop(0)
        stories.append(line)

In [ ]:
stories_with_candidates = []
for orig_story in stories:
    story_length = len(orig_story) - 1
    story_and_candidates = []
    #print(orig_story[0])
    for sentence in orig_story:
        candidates = []
        #print ("BEFORE: ", orig_story)
        sent1 = random.randint(0, story_length)
        sent2 = random.randint(0, story_length)
        sent3 = random.randint(0, story_length)
        sent4 = random.randint(0, story_length)
        story1 = random.randint(0, len(stories) -1)
        story2 = random.randint(0, len(stories) -1)
        story3 = random.randint(0, len(stories) -1)
        story4 = random.randint(0, len(stories) -1)
        candidates.append(sentence)
        candidates.append(stories[story1][sent1])
        candidates.append(stories[story2][sent2])
        candidates.append(stories[story3][sent3])
        candidates.append(stories[story4][sent4])
        story_and_candidates.append(candidates)
        #print ("AFTER: ", orig_story)
    stories_with_candidates.append(story_and_candidates)

In [ ]:
print(stories_with_candidates)

In [ ]:
def story_compatability(story):
    result = 1.
    for i in range(len(story)-1):
        sent_a = story[i]
        sent_b = story[i+1]
        encoded = tokenizer.encode_plus(sent_a, sent_b, return_tensors='pt').to(device)
        seq_relationship_logits = model(**encoded)[0]
        probs = softmax(seq_relationship_logits, dim=1)
        compt = probs[0][0].tolist()
        result *= compt
    return result

In [ ]:
from tqdm import tqdm
result1 = 0
result2 = 0
for story_one in tqdm(stories_with_candidates[:(len(stories) -1)]):
    #print("Story ", story_one)
    #input()
    #shuf_story = story_one.copy()
    #random.shuffle(shuf_story)
    prob1 = story_compatability(story_one)
    #if prob1 < 0.1:
        #print("PROB1 ", story_one, " ", prob1)
    #print(prob)
    #shuf_prob = story_compatability(shuf_story)
    #print(shuf_prob)
    prob2 = story_compatability_3(story_one)
    #if prob2 < 0.1:
        #print("PROB2 ", story_one, " ", prob2)
    result1 += prob1
    result2 += prob2
    #print(prob1)
    #print(prob2)
    #shuf_prob = story_compatability_3(shuf_story)
    #print(shuf_prob)
print (result1/(len(stories) -1))
print (result2/(len(stories) -1))